# 🧠 Multi-LLM Brochure Generator (GPT, Claude, LLaMA)
Generate a company brochure using OpenAI, Anthropic, and locally hosted LLaMA 3.2 models — all streaming side-by-side in real time using Gradio.


In [ ]:
!pip install openai google-generativeai anthropic ollama gradio requests beautifulsoup4 python-dotenv

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import gradio as gr
import threading
from queue import Queue
from dotenv import load_dotenv

import openai
import anthropic
import google.generativeai
import ollama

load_dotenv(override=True)

# API setup
openai.api_key = os.getenv("OPENAI_API_KEY")
claude = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
google.generativeai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

system_message = "You are a helpful AI assistant that generates compelling company brochures."


In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for tag in soup.body(["script", "style", "img", "input"]):
            tag.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [ ]:
def stream_gpt(prompt, queue):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        stream=True
    )
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            queue.put(("GPT", content))
    queue.put(("GPT", None))

def stream_claude(prompt, queue):
    result = claude.messages.stream(
        model="claude-sonnet-20240229",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[{"role": "user", "content": prompt}],
    )
    with result as stream:
        for text in stream.text_stream:
            if text:
                queue.put(("Claude", text))
    queue.put(("Claude", None))

def stream_llama(prompt, queue):
    stream = ollama.chat(
        model="llama3:latest",
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        stream=True
    )
    for chunk in stream:
        content = chunk['message']['content']
        if content:
            queue.put(("LLaMA", content))
    queue.put(("LLaMA", None))


In [ ]:
def stream_gpt(prompt, queue):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        stream=True
    )
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            queue.put(("GPT", content))
    queue.put(("GPT", None))

def stream_claude(prompt, queue):
    result = claude.messages.stream(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[{"role": "user", "content": prompt}],
    )
    with result as stream:
        for text in stream.text_stream:
            if text:
                queue.put(("Claude", text))
    queue.put(("Claude", None))

def stream_llama(prompt, queue):
    try:
        stream = ollama.chat(
            model="llama3.2",  # or your preferred tag
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ],
            stream=True
        )

        for chunk in stream:
            content_piece = chunk['message']['content']
            if content_piece:
                queue.put(("LLaMA", content_piece))
    except Exception as e:
        queue.put(("LLaMA", f"\n[Error using Ollama client: {str(e)}]\n"))
    finally:
        queue.put(("LLaMA", None))


In [ ]:
def stream_worker(company_name, url, model, queue):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model == "GPT":
        stream_gpt(prompt, queue)
    elif model == "Claude":
        stream_claude(prompt, queue)
    elif model == "LLaMA":
        stream_llama(prompt, queue)


In [ ]:
def stream_brochures(company_name, url, selected_models):
    output = {"GPT": "", "Claude": "", "LLaMA": ""}
    queue = Queue()
    threads = []

    for model in selected_models:
        t = threading.Thread(target=stream_worker, args=(company_name, url, model, queue))
        t.start()
        threads.append(t)

    finished_models = set()
    while len(finished_models) < len(selected_models):
        model, chunk = queue.get()
        if chunk is None:
            finished_models.add(model)
            continue
        output[model] += chunk
        yield output.get("GPT", ""), output.get("Claude", ""), output.get("LLaMA", "")

    for t in threads:
        t.join()


In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        company_input = gr.Textbox(label="Company name:")
        url_input = gr.Textbox(label="Landing page URL")
        model_input = gr.CheckboxGroup(["GPT", "Claude", "LLaMA"], label="Select model(s)")

    run_btn = gr.Button("Generate Brochures")

    with gr.Row():
        with gr.Column():
            gr.Markdown("### GPT Brochure")
            output_gpt = gr.Markdown()
        with gr.Column():
            gr.Markdown("### Claude Brochure")
            output_claude = gr.Markdown()
        with gr.Column():
            gr.Markdown("### LLaMA Brochure")
            output_llama = gr.Markdown()

    run_btn.click(
        fn=stream_brochures,
        inputs=[company_input, url_input, model_input],
        outputs=[output_gpt, output_claude, output_llama]
    )

demo.launch()
